In [1]:
import numpy as np 

from scipy.optimize import linprog

from itertools import combinations

from colorama import Fore, Back, Style

import math as m

In [2]:
def all_diag_combination(dim): 

    possible_combination = set(combinations([-1, 1] * dim ,dim))

    return list(np.diag(item) for item in possible_combination)

In [3]:
dim = int(input("Enter M: ")) 

din = int(input("Enter N: "))

 

################################# input matrix A ################################

 

print("Enter Matrix A :")                           

Ac = []              #center of matrix A

Au = []              #upper bound of matrix A 

delta = []           #radius of matrix A

A = []               #matrix A

for m in range(dim):

    Ac_1 = []

    Au_1 = []

    delta_1 = []
    
    a_1 = []

    for n in range(din):
        
        a_2 = []

        value_A = input("Enter 2 numbers (with a comma in between): ")                  #input matrix A in the pattern of (a, b)

        a, b = value_A.split(',')                                                       #splits a and b

        a, b = float(a), float(b)                                                       #transfrom str(a), str(b) to float(a), float(b) respectively

        Ac_1.append((a+b)*0.5)                                                                                      

        Au_1.append(b)                                                                  #add b(upper bound) in upper bound matrix

        delta_1.append((b-a)*0.5)
        
        a_2.append(a)
        
        a_2.append(b)
        
        a_1.append(a_2)

    Ac.append(Ac_1)

    Au.append(Au_1)

    delta.append(delta_1)
    
    A.append(a_1)

print("Matrix A is", A)   

print("Center matrix of A =", Ac)

print("Upper bound matrix of A =", Au)

print("Radius matrix of A =", delta, '\n')

 

################################# input matrix B ################################

 

print("Enter Matrix B :")

Bc = []         #center of matrix b

Bl = []         #lower bound of matrix b

tuda = []       #radius of matrix b

B = []          #matrix B

for m in range(dim):
    
    b_1 = []

    value_B = input("Enter 2 numbers (with a comma in between): ")

    c, d = value_B.split(',')

    c, d = float(c), float(d)

    Bc.append((c+d)*0.5)

    Bl.append(c)

    tuda.append((d-c)*0.5)
    
    b_1.append(c)
    
    b_1.append(d)
    
    B.append(b_1)
   
print("Matrix B is", B)
    
print("Center matrix of B =", Bc)

print("Lower bound matrix of B =", Bl)

print('Radius matrix of B =', tuda, '\n')

 

################################# input matrix C ################################

 

print("Enter Matrix C :")

C = []             #matrix c

for r in range(din):

    value_C = float(input("Enter number: "))

    C.append(value_C)

   

print("Matrix C =", C)

Enter M: 2
Enter N: 2
Enter Matrix A :
Enter 2 numbers (with a comma in between): 4,6
Enter 2 numbers (with a comma in between): -8,-6
Enter 2 numbers (with a comma in between): 9,11
Enter 2 numbers (with a comma in between): 11,13
Matrix A is [[[4.0, 6.0], [-8.0, -6.0]], [[9.0, 11.0], [11.0, 13.0]]]
Center matrix of A = [[5.0, -7.0], [10.0, 12.0]]
Upper bound matrix of A = [[6.0, -6.0], [11.0, 13.0]]
Radius matrix of A = [[1.0, 1.0], [1.0, 1.0]] 

Enter Matrix B :
Enter 2 numbers (with a comma in between): -10,-8
Enter 2 numbers (with a comma in between): 19,21
Matrix B is [[-10.0, -8.0], [19.0, 21.0]]
Center matrix of B = [-9.0, 20.0]
Lower bound matrix of B = [-10.0, 19.0]
Radius matrix of B = [1.0, 1.0] 

Enter Matrix C :
Enter number: 1
Enter number: 3
Matrix C = [1.0, 3.0]


In [4]:
bnd = [] 

p_inf = float("inf")

for e in range(din):
    
    bnd.append((0, p_inf))
    
print(bnd)

[(0, inf), (0, inf)]


In [5]:
###................................. Checking strong feasibility .................................### 

res_1 = linprog(C, A_ub=Au, b_ub=Bl, method='revised simplex')

print(res_1, '\n')

if res_1['status']==0:

    print('This interval linear programming is strongly feasible.', '\n')
    
elif res_1['status']==2:
    
    print( 'This interval linear programming is not strongly feasible.', '\n')
    
######################################################################################################    
    
    
    
###...................................... Lower bound case .......................................###    

delta_2 = delta/(np.ones((dim,din))*10)

tuda_2 = tuda/(np.ones((dim))*10)

Au_2 = Ac - delta_2

Al_2 = Ac + delta_2

Bu_2 = Bc - tuda_2

Bl_2 = Bc + tuda_2

fun_2 = []

status = []

for i in range(11):

    Au_2 += delta_2

    Al_2 -= delta_2

    Bu_2 += tuda_2

    Bl_2 -= tuda_2

    A = np.append( arr=Al_2, values=Au_2, axis=0 )

    B = np.append( arr=Bu_2, values=Bl_2, axis=0 )

    res = linprog(C, A_ub = A, b_ub = B, bounds=bnd, method = 'revised simplex' )

    status.append(res['status'])

    print('i =', i)

    if 1 in status:

        print('Iteration limit reached', '\n')

    elif 2 in status:

        print('Lower Bound:', 'Problem appears to be infeasible', '\n')

    elif 3 in status:

        print('Lower Bound:', 'Problem appears to be unbounded', '\n')

    elif 4 in status:

        print('Numerical difficulties encountered', '\n')

    elif 5 in status:

        print('No constraints; turn presolve on', '\n')

    elif 6 in status:

        print('Guess x0 cannot be converted to a basic feasible solution', '\n')

    else:

        fun_2.append(res['fun'])

        print('Lower bound is', '%.4f' %res['fun'])

        print('x is', res['x'], '\n')
 

#print(fun_2, '\n')

##############################################################################################################



###................................................ Upper bound case ......................................###

for j in range(11):

    Status = []

    Fun = []
    
    X = []

    X_3 = []
    
    for r in range(din):
        
        X_1 = []
        
        X_2 = []

        Res1 = []     

        aye = Ac - (all_diag_combination(dim)[r]).dot(delta/(np.ones((dim,din))*10)*(np.ones((dim,din))*j))

        aye = np.array(aye)

        By = Bc + (all_diag_combination(dim)[r]).dot(tuda/(np.ones((dim))*10)*(np.ones((dim))*j))

        #bnd = [(0, float("inf")), (0, float("inf")), (0, float("inf")), (0, float("inf"))]

        Res = linprog(c=C, A_eq=aye, b_eq=By, bounds=bnd, method="revised simplex")

        Status.append(Res['status'])
        
        X_2.append(Res['fun'])       #contain both all optimal value and optimal solution for each case (i)
        
        X_2.append(Res['x'])         #contain both all optimal value and optimal solution for each case (i)
        
        X_1.append(X_2)              #[[optimal value, optimal solution]] for each case i
        
        X.append(X_1)                #[[optimal value, optimal solution],...] for all case i
        
        X_3.append(Res['x'])         #contain only all optimal solution for each case (i) 

    print('i =', j)

    if 1 in Status:

        print('Iteration limit reached', '\n')

    elif 2 in Status:

        print('Upper Bound:', 'Problem appears to be infeasible', '\n')

    elif 3 in Status:

        print('Upper Bound:', 'Problem appears to be unbounded', '\n')

    elif 4 in Status:

        print('Numerical difficulties encountered', '\n')

    elif 5 in Status:

        print('No constraints; turn presolve on', '\n')

    elif 6 in Status:

        print('Guess x0 cannot be converted to a basic feasible solution', '\n')

    else:
        
        print('All possible optimal value is',Fun)
        
        X = sorted(X,key=lambda l:l[0][0], reverse=True)         #sort all possible optimal value
        
        #print(X)
        
        #print(X[0][0][0], X[0][0][1])
        
        print('All possible optimal solution x is ', X_3)

        print('Then upper bound is', '%.4f' %X[0][0][0], 'where x is ', X[0][0][1], '\n')

     con: array([], dtype=float64)
     fun: 4.384615384615384
 message: 'The problem appears infeasible, as the phase one auxiliary problem terminated successfully with a residual of 1.2e+00, greater than the tolerance 1e-12 required for the solution to be considered feasible. Consider increasing the tolerance to be greater than 1.2e+00. If this tolerance is unnaceptably large, the problem is likely infeasible.'
     nit: 1
   slack: array([-1.23076923,  0.        ])
  status: 2
 success: False
       x: array([0.        , 1.46153846]) 

This interval linear programming is not strongly feasible. 

i = 0
Lower bound is 3.8571
x is [0.         1.28571429] 

i = 1
Lower bound is 3.9565
x is [0.         1.31884058] 

i = 2
Lower bound is 4.0588
x is [0.         1.35294118] 

i = 3
Lower bound is 4.1642
x is [0.        1.3880597] 

i = 4
Lower bound is 4.2727
x is [0.         1.42424242] 

i = 5
Lower bound is 4.3846
x is [0.         1.46153846] 

i = 6
Lower bound is 4.5000
x is [0.  1.5]